In [1]:
import nltk
from collections import defaultdict, Counter
import gensim, logging
import nltk
from gensim import matutils
from gensim.models.keyedvectors import KeyedVectors
from gensim.similarities.index import AnnoyIndexer
from multiprocessing import Process, Pool
import os
import collections
from random import shuffle
from copy import deepcopy
from collections import OrderedDict
import pickle
import networkx as nx
import matplotlib.pyplot as plt
from operator import itemgetter
from scipy.stats import spearmanr
import pandas as pd
import numpy as np
from gensim.models.wrappers import FastText
import string
from numpy import dot, zeros, dtype, float32 as REAL,\
    double, array, vstack, fromstring, sqrt, newaxis,\
    ndarray, sum as np_sum, prod, ascontiguousarray,\
    argmax

#### Spearman Correlation for different word embeddings on Standford rare word dataset
word2vec: (0.2516646845987402)

glove6B 50dimensions (0.26159587465252532, 1782)

glove 100dimensions (0.2136083090117197, 1782)

glove 200dimensions (0.17667532980978479, 1782)

glove 300dimensions (0.15571816294824803, 1782)

glove 42B 300dimensions 

fasttext

In [2]:
%time fasttext =KeyedVectors.load_word2vec_format('/home/raja/models/GoogleNews-vectors-negative300.bin', binary=True)

CPU times: user 25.4 s, sys: 2.56 s, total: 27.9 s
Wall time: 31 s


In [3]:
graph_file = '../data/norm_graph_100000_15390'
graph_handle = open(graph_file, "rb")
G = pickle.load(graph_handle)
graph_handle.close()

In [4]:
list(G.edges(keys=True, data=True))[38]

('dairies',
 'dairy',
 ('currencies', 'currency'),
 {'cos': 0.541899710893631, 'morp_rule': ('suffix', 'ies', 'y'), 'rank': 2})

In [5]:
morph_rules = {}
for edge in G.edges(keys=True, data=True):
    morph_rules[edge[3]['morp_rule']] = edge[2]

In [14]:
def get_spearman_corr(word_vectors):
    rw = pd.read_csv('../dataset/rw/rw.txt', header=None,sep='\t')
    rare_word_pair = []
    for index, row in rw.iterrows():
        rare_word_pair.append((row[0], row[1]))
    corr_list = []
    for word1,word2 in rare_word_pair:
        try:
            cor, p = spearmanr(word_vectors.wv[word1], word_vectors.wv[word2])
            corr_list.append(cor)
        except Exception as e:
            if word1 in word_vectors.vocab:
                match, word2_vec = get_oov_vector(word2)
                if match:
                    cor, p = spearmanr(word_vectors.wv[word1],word2_vec)
                    corr_list.append(cor)
            elif word2 in word_vectors.vocab:
                match, word1_vec = get_oov_vector(word1)
                if match:
                    cor, p = spearmanr(word_vectors.wv[word2],word1_vec)
                    print (cor)
                    corr_list.append(cor)
    return np.mean(corr_list), len(corr_list)
    # w2v 0.2516646845987402

In [12]:
def get_oov_vector(word):
    new_word = (False, 0, ())
    mean = []
    for rule in morph_rules:
        candidate_word = apply_rule(rule, word)
        if candidate_word and candidate_word in fasttext.vocab:
            if fasttext.vocab[candidate_word].count > new_word[1]:
                new_word = (candidate_word, fasttext.vocab[candidate_word].count, morph_rules[rule])
    if new_word[0]:
#         print (word, new_word)
        mean.append(fasttext[new_word[2][0]] * 1)
        mean.append(fasttext[new_word[2][1]] * -1)
        mean.append(fasttext[new_word[0]] * 1)
        mean = matutils.unitvec(array(mean).mean(axis=0)).astype(REAL)
        return True, mean
    else:
        return False, []
#             print ("new word",new_word, word)

In [9]:
def apply_rule(rule, word):
    if rule[0] == 'suffix':
        if word[::-1].startswith(rule[1][::-1]):
            return word[::-1].replace(rule[1][::-1], rule[2][::-1], 1)[::-1]
    else:
        if word.startswith(rule[1]):
            return word.replace(rule[1], rule[2], 1)
    return False

In [15]:
get_spearman_corr(fasttext)

0.145091547164
0.0586183103054
0.461028017136
0.194759020851
0.409499486385
0.281615950301
0.127427707669
-0.0375941445307
0.0426986479765
0.0772181758592
0.207170483829
0.152037645073
0.104718809883
0.206542086088
0.153295845285
0.146953546801
0.11964886023
0.220636231242
-0.0709737936544
0.138300764825
0.559380963295
0.0269203824295
0.16240955395
0.120867485763
0.0302293387003
0.10571231203
0.171614333647
0.258678993413
0.089690934829
0.0224695617624
0.0818646067835
0.225973336788
0.313721257011
0.35424216562
0.061266819314
0.270940436853
0.310300673635
0.349981838582
0.130946144073
0.300886236995
0.101310879881
0.0637571878996
0.378034913045
0.418296547497
0.149389662254
0.219980321906
0.231326358785
0.0343664540263
0.0647884059417
0.132478792977
0.184991025401
0.149451946787
0.101028939715
0.0703118133558
0.718936388932
0.0837345444382
0.0489405802248
0.178048987267
0.136317145582
0.0992075771787
0.0407372661947
0.0167565405967
0.114585811385
0.109230275206
0.132547823597
0.4763793

(0.24562283855541628, 1988)